In [1]:
'''
plt.figure(figsize=(26, 6))
data['Adj Close'].plot(x = 250, y = 10, linewidth = 0.8)
plt.show()

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
acf_original = plot_acf(data['Adj Close'])
pacf_original = plot_pacf(data['Adj Close'])

# adf test
from statsmodels.tsa.stattools import adfuller
adf_result = adfuller(data['Adj Close'])
print("ADF Statistic:", adf_result[1])
# fails to reject null hypothesis at 5% sig level, non-stationary

transforming to stationary (stabilise mean)
df_train_diff = data['Adj Close'].diff().dropna()
df_train_diff.plot()

acf_diff = plot_acf(df_train_diff)
pacf_diff = plot_pacf(df_train_diff)

adf_result_diff = adfuller(df_train_diff)
print("ADF Statistic (differenced):", adf_result_diff[1])
# reject null hypothesis at a 5% significance level, stationary

# make time series predictions

import matplotlib.pyplot as plt
residuals = model_fit.resid[1:]
fig, ax = plt.subplots(1,2)
residuals.plot(title = "Residuals", ax = ax[0])
residuals.plot(kind = 'kde', title = "Density", ax = ax[1])
plt.show()


plot_acf(residuals)
plot_pacf(residuals)
plt.show()
'''
'''
import pmdarima as pm
import os
import pandas as pd

clean_path = os.path.join("..", "data", "processed")
tickers = [ticker for ticker in os.listdir(clean_path) if ticker.endswith('.csv')]
forecast_results = {}

# Forecasting next n days for each ticker
for ticker in tickers:
    file_path = os.path.join(clean_path, ticker)
    cols = pd.read_csv(file_path, nrows=0).columns
    if ticker == "import_data.csv":
        continue
    data = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date", low_memory=False)
    data = data.asfreq('B').ffill()
    returns = data["Return"].dropna()
    model = pm.auto_arima(returns, start_p=0, d=None, start_q=0, max_p=3, max_q=3, seasonal=False, stepwise=True, suppress_warnings=True, n_jobs=1)
    forecast_vals = model.predict(steps=10)
    forecast_i = pd.bdate_range(start=returns.index[-1] + pd.offsets.BDay(1), periods=10, freq='B')
    forecast = pd.Series(forecast_vals, index=forecast_i)
    forecast_results[ticker] = forecast
    print("finished!")
print(forecast_results)

# Saving the forecast results to a CSV file 
forecast_df = pd.DataFrame(forecast_results)
output_path = os.path.join("..", "data", "forecasts", "forecasted_returns.csv")
forecast_df.to_csv(output_path)  
    
# Getting the diagram for how it looks
# Printing a prediction for whether or not its a good idea to invest or not
'''

'\nimport pmdarima as pm\nimport os\nimport pandas as pd\n\nclean_path = os.path.join("..", "data", "processed")\ntickers = [ticker for ticker in os.listdir(clean_path) if ticker.endswith(\'.csv\')]\nforecast_results = {}\n\n# Forecasting next n days for each ticker\nfor ticker in tickers:\n    file_path = os.path.join(clean_path, ticker)\n    cols = pd.read_csv(file_path, nrows=0).columns\n    if ticker == "import_data.csv":\n        continue\n    data = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date", low_memory=False)\n    data = data.asfreq(\'B\').ffill()\n    returns = data["Return"].dropna()\n    model = pm.auto_arima(returns, start_p=0, d=None, start_q=0, max_p=3, max_q=3, seasonal=False, stepwise=True, suppress_warnings=True, n_jobs=1)\n    forecast_vals = model.predict(steps=10)\n    forecast_i = pd.bdate_range(start=returns.index[-1] + pd.offsets.BDay(1), periods=10, freq=\'B\')\n    forecast = pd.Series(forecast_vals, index=forecast_i)\n    forecast_results[tic

In [3]:
import os
import pandas as pd, numpy as np
import pmdarima as pm

clean_path = os.path.join("..", "data", "processed") 

def forecast_ticker(ticker, n_steps):
    path = os.path.join("..", "data", "processed") 
    file_path = os.path.join(path, f"{ticker}.csv")
    df = pd.read_csv(file_path, parse_dates=["Date"], index_col="Date", low_memory=False).asfreq("B").ffill()

    price_col = "Adj Close"

    returns = df["Return"].dropna()
    last_price = float(df[price_col].iloc[-1])

    model = pm.auto_arima(returns, start_p=0, start_q=0, max_p=3, max_q=3, d=None, seasonal=False, stepwise=True, suppress_warnings=True, n_jobs=1)

    vals, confi = model.predict(n_periods=int(n_steps), return_conf_int=True, alpha=0.05)
    idx = pd.bdate_range(start=returns.index[-1] + pd.offsets.BDay(1), periods=int(n_steps), freq="B")

    fc_returns = pd.DataFrame({"ret_hat": vals, "ret_lower": confi[:,0], "ret_upper": confi[:,1]}, index=idx)
    fc_returns.index.name = "Date"

    cumu = (1.0 + pd.Series(fc_returns["ret_hat"].values, index=idx)).cumprod()
    fc_price = pd.DataFrame({"yhat": last_price * cumu}, index=idx)
    fc_price.index.name = "Date"
    return fc_returns, fc_price

# quick test
TICKER, N = "AAPL", 10
ret, px = forecast_ticker(TICKER, N)
print(ret)
print(px)

             ret_hat  ret_lower  ret_upper
Date                                      
2024-12-31  0.001854  -0.032823   0.036532
2025-01-01  0.001043  -0.033685   0.035771
2025-01-02  0.001043  -0.033685   0.035771
2025-01-03  0.001043  -0.033685   0.035771
2025-01-06  0.001043  -0.033685   0.035771
2025-01-07  0.001043  -0.033685   0.035771
2025-01-08  0.001043  -0.033685   0.035771
2025-01-09  0.001043  -0.033685   0.035771
2025-01-10  0.001043  -0.033685   0.035771
2025-01-13  0.001043  -0.033685   0.035771
                  yhat
Date                  
2024-12-31  251.773917
2025-01-01  252.036476
2025-01-02  252.299309
2025-01-03  252.562416
2025-01-06  252.825798
2025-01-07  253.089454
2025-01-08  253.353385
2025-01-09  253.617591
2025-01-10  253.882073
2025-01-13  254.146830


In [ ]:
import riskfolio as rf
port = rf.Portfolio(returns=[1,2,3])
print(port)